In [ ]:
import vpython
import numpy as np
import scipy.linalg
import math
import mpmath
import qutip
import random

class Sphere:
    def __init__(self, n):
        self.n = n
        self.state = qutip.rand_ket(self.n)
        self.energy = qutip.rand_herm(self.n)
        


def scalar_to_vector(z):
    x = z.real
    y = z.imag
    return [(2*x)/(1.+(x**2)+(y**2)),\
           (2*y)/(1.+(x**2)+(y**2)),\
           (-1.+(x**2)+(y**2))/(1.+(x**2)+(y**2))]

def vector_to_scalar(x, y, z):
    if z == 1:
        return Inf 
    else:
        return complex(x/(1-z), y/(1-z))

# Dimensionality
n = 4
# Total pure state vector of the sphere
state = qutip.rand_ket(n)
# Total energy function
energy = qutip.rand_herm(n)
# Graphics init
vpython.scene.range = 1
vpython.scene.forward = vpython.vector(random.random(), random.random(), random.random())
sphere = vpython.sphere(pos=vpython.vector(0,0,0), radius=1,\
                     color=vpython.color.blue, opacity=0.5)

# Emanations...
polynomial = None
pure_scalars = None
pure_vectors = None
pure_matrices = None
m = None
impure_matrices = None
impure_vectors = None
pure_stars = None
impure_stars = None

def emanate(state):
    global polynomial
    global pure_scalars
    global pure_vectors
    global pure_matrices
    global m
    global impure_matrices
    global impure_vectors
    global pure_stars
    global impure_stars
    polynomial = state.full().T.tolist()[0]
    # Roots of corresponding polynomial on equatorial plane
    pure_scalars = mpmath.polyroots(polynomial)
    # Spacetime vectors to surface stars on sphere
    pure_vectors = [scalar_to_vector(scalar) for scalar in pure_scalars]
    # Hermitian matrices of surface stars
    pure_matrices = [vector[0]*qutip.sigmax().full()+\
                         vector[1]*qutip.sigmay().full()+\
                         vector[2]*qutip.sigmaz().full() for vector in pure_vectors]
    # Hermitian matrices of interior stars
    m = int(math.log(n,2))
    state.dims = [[2]*m,[1]*m]
    impure_matrices = [state.ptrace(i) for i in range(m)]
    # Spacetime vectors of interior stars
    impure_vectors = [[qutip.expect(qutip.sigmax(), matrix),\
                       qutip.expect(qutip.sigmay(), matrix),\
                       qutip.expect(qutip.sigmaz(), matrix)] 
                            for matrix in impure_matrices]
    if pure_stars == None:
        pure_stars = [vpython.sphere(pos=vpython.vector(*vector),\
                             radius=0.1, color=vpython.color.white,\
                             opacity=1.0, emissive=False) for vector in pure_vectors]
    else:
        for i in range(len(pure_stars)):
            pure_stars[i].pos = vpython.vector(*pure_vectors[i])
    if impure_stars == None:
        impure_stars = [vpython.sphere(pos=vpython.vector(*vector),\
                               radius=0.1, color=vpython.color.red,\
                               opacity=1.0, emissive=False) for vector in impure_vectors]
    else:
        for i in range(len(impure_stars)):
            impure_stars[i].pos = vpython.vector(*impure_vectors[i])

t = 0
dt = 0.0002
emanate(state)
while True:
    vpython.rate(10)
    propagator = scipy.linalg.expm(-2*math.pi*complex(0,1)*energy.full()*t)
    state = qutip.Qobj(propagator.dot(state.full()))
    emanate(state)
    t += dt

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>